In [265]:
import pandas as pd

In [266]:
data = pd.read_csv('train.csv')

In [267]:
data.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0


In [268]:
data.isnull().sum()

id                 0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [269]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165034 entries, 0 to 165033
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               165034 non-null  int64  
 1   CustomerId       165034 non-null  int64  
 2   Surname          165034 non-null  object 
 3   CreditScore      165034 non-null  int64  
 4   Geography        165034 non-null  object 
 5   Gender           165034 non-null  object 
 6   Age              165034 non-null  float64
 7   Tenure           165034 non-null  int64  
 8   Balance          165034 non-null  float64
 9   NumOfProducts    165034 non-null  int64  
 10  HasCrCard        165034 non-null  float64
 11  IsActiveMember   165034 non-null  float64
 12  EstimatedSalary  165034 non-null  float64
 13  Exited           165034 non-null  int64  
dtypes: float64(5), int64(6), object(3)
memory usage: 17.6+ MB


In [270]:
data = data.drop(columns=["id", "CustomerId", "Surname"])

In [271]:
x = data[data.columns[:-1]]
y = data[data.columns[-1]]

In [272]:
# bins = [18, 30, 40, 50, 60, 70, 92]
# labels = ['18-29', '30-39', '40-49', '50-59', '60-69', '70+']

# x['AgeCategory'] = pd.cut(x['Age'], bins=bins, labels=labels, include_lowest=True, right=False)


In [273]:
x.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97
1,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50
2,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69
3,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88
4,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83


In [274]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
encoder = OneHotEncoder()
categorical_columns = ["Geography","Gender"]

for column in categorical_columns:
    categorical = pd.DataFrame(x[column])
    transformed = encoder.fit_transform(categorical)
    transformed_df = pd.DataFrame(transformed.toarray(), columns=encoder.get_feature_names_out([column]))
    x = pd.concat([x, transformed_df], axis=1)
    x.drop(columns=[column], inplace=True)



In [275]:
# le = LabelEncoder()
# x["Gender"] = le.fit_transform(x["Gender"])

In [276]:
x.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,668,33.0,3,0.00,2,1.0,0.0,181449.97,1.0,0.0,0.0,0.0,1.0
1,627,33.0,1,0.00,2,1.0,1.0,49503.50,1.0,0.0,0.0,0.0,1.0
2,678,40.0,10,0.00,2,1.0,0.0,184866.69,1.0,0.0,0.0,0.0,1.0
3,581,34.0,2,148882.54,1,1.0,1.0,84560.88,1.0,0.0,0.0,0.0,1.0
4,716,33.0,5,0.00,2,1.0,1.0,15068.83,0.0,0.0,1.0,0.0,1.0


In [277]:
y.value_counts()

Exited
0    130113
1     34921
Name: count, dtype: int64

Handling Data Imbalance

In [278]:
from imblearn.over_sampling import RandomOverSampler

randomOverSampler = RandomOverSampler()

x_resampled, y_resampled = randomOverSampler.fit_resample(x,y)

In [279]:
y_resampled.value_counts()

Exited
0    130113
1    130113
Name: count, dtype: int64

In [280]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x_resampled)

In [281]:
# from sklearn.feature_selection import mutual_info_classif
# importances = mutual_info_classif(x_scaled,y_resampled)
# feat_importance = pd.Series(importances, data.columns[0:len(data.columns)-1])
# feat_importance.plot(kind='barh', color = 'teal')
# plt.show()

In [282]:
from sklearn.model_selection import train_test_split
x_latih, x_uji, y_latih, y_uji = train_test_split(x_scaled,y_resampled ,test_size=0.2, random_state=2)

In [283]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV 

from scipy.stats import randint as sp_randint

param_dist = {"max_depth": [3, None],
              "max_features": sp_randint(1, 11),
              "min_samples_split": sp_randint(2, 11),
              "min_samples_leaf": sp_randint(1, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

rf = RandomForestClassifier(max_features=10, criterion='entropy')
# grid = GridSearchCV(rf, param_dist, cv=5) 


In [284]:
rf.fit(x_latih, y_latih)

print("Akurasi Model RF pada Data Uji:", rf.score(x_uji,y_uji))
# print('best params', rf.best_params_)

Akurasi Model RF pada Data Uji: 0.9380932252238404


In [285]:
data_test = pd.read_csv('test.csv')

In [286]:
id = data_test["id"]
data_test = data_test.drop(columns=['id','CustomerId', 'Surname'])

In [287]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
categorical_columns = ["Geography", "Gender"]

for column in categorical_columns:
    categorical = pd.DataFrame(data_test[column])
    transformed = encoder.fit_transform(categorical)
    transformed_df = pd.DataFrame(transformed.toarray(), columns=encoder.get_feature_names_out([column]))
    data_test = pd.concat([data_test, transformed_df], axis=1)
    data_test.drop(columns=[column], inplace=True)

In [288]:
data_test.isna().sum()

CreditScore          0
Age                  0
Tenure               0
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Geography_France     0
Geography_Germany    0
Geography_Spain      0
Gender_Female        0
Gender_Male          0
dtype: int64

In [289]:
data_scaled = scaler.fit_transform(data_test)

In [290]:
hasil = rf.predict(data_scaled)

In [291]:
hasil = pd.Series(hasil)

In [292]:
hasil = pd.concat([id, hasil], axis=1)

In [293]:
hasil.head()

,id,0
0,165034,0
1,165035,1
2,165036,0
3,165037,0
4,165038,1


In [294]:
hasil.rename(columns = {0:'Exited'}, inplace=True)

In [295]:
hasil.to_csv('submission_rf.csv', index=False)